In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import os

### def

In [6]:
# Define function to calculate ECF
def calculate_ecf(adata, genes, cell_type, cell_type_col, exclude_type=None):
    gene_ecf_list = []
    for gene_name in genes:
        if gene_name in adata.var_names:
            gene_expression = adata[:, gene_name].X.toarray().flatten()
            expressed_cells = np.sum(gene_expression > 0)
            total_cells = gene_expression.shape[0]

            # Exclude certain cell type if specified
            if exclude_type:
                excluded_cells_count = np.sum(adata.obs[cell_type_col] == exclude_type)
                adjusted_total_cells = total_cells - excluded_cells_count
                ecf = (expressed_cells / adjusted_total_cells) * 100 if adjusted_total_cells > 0 else 0
            else:
                ecf = (expressed_cells / total_cells) * 100

            gene_ecf_list.append({
                'Gene': gene_name,
                f'{cell_type}_ECF': ecf
            })
        else:
            gene_ecf_list.append({
                'Gene': gene_name,
                f'{cell_type}_ECF': np.nan
            })
    return pd.DataFrame(gene_ecf_list)

# Define function to calculate cell type percentage
def calculate_cell_type_percentage(genes, adata, cell_types, cell_type_col):
    data = []
    for gene_name in genes:
        gene_data = {'Gene': gene_name}
        if gene_name in adata.var_names:
            gene_expression = adata[:, gene_name].X.toarray().flatten()
            expressed_cells = adata[gene_expression > 0, :]

            for cell_type in cell_types:
                total_cells_in_type = np.sum(adata.obs[cell_type_col] == cell_type)
                expressed_cells_in_type = np.sum(expressed_cells.obs[cell_type_col] == cell_type)
                percentage = (expressed_cells_in_type / total_cells_in_type) * 100 if total_cells_in_type > 0 else 0
                gene_data[cell_type] = round(percentage, 2)
        else:
            for cell_type in cell_types:
                gene_data[cell_type] = np.nan
        data.append(gene_data)
    
    return pd.DataFrame(data)

# Define function to create empty dataframe
def create_empty_df(genes, cell_types):
    return pd.DataFrame(0.0, index=genes, columns=cell_types)

# Define function to calculate cell type percentage among expressed cells
def calculate_exp_cell_type_percentage(genes, adata, cell_types, cell_type_col):
    exp_cell_type_percentage_df = create_empty_df(genes, cell_types)
    for gene_name in genes:
        if gene_name in adata.var_names:
            gene_expression = adata[:, gene_name].X
            expressed_cells = adata[gene_expression > 0, :]

            if expressed_cells.n_obs > 0:
                cell_type_counts = expressed_cells.obs[cell_type_col].value_counts(normalize=True) * 100

                for cell_type, percentage in cell_type_counts.items():
                    exp_cell_type_percentage_df.loc[gene_name, cell_type] = round(percentage, 2)
    
    exp_cell_type_percentage_df.fillna(0, inplace=True)
    return exp_cell_type_percentage_df

## Load adata

In [27]:
# no prepro data
adata_gs = sc.read('data/CCA_Lung_geosketch.h5ad')

adata_gs_tumor = adata_gs[adata_gs.obs['cell_type_subset'] == 'tumor'].copy()
adata_gs_Tip_Cells = adata_gs[adata_gs.obs['cell_type_subset'] == 'Tip_Cells'].copy()

In [28]:
# prepro data
adata_pre = sc.read('data/CCA_Lung_geosketch_prepro.h5ad')

adata_pre_tumor = adata_pre[adata_pre.obs['cell_type_subset'] == 'tumor'].copy()
adata_pre_Tip_Cells = adata_pre[adata_pre.obs['cell_type_subset'] == 'Tip_Cells'].copy()

In [43]:
# Load result files
path = 'res/'
dfs = {}
file_list = [f for f in os.listdir(path) if f.startswith("gcam_") and f.endswith(".csv")]

for file in file_list:
    key = file.split("gcam_")[1].split("_res")[0]  # Extract the part of the string between 'gcam_' and '._res'
    df = pd.read_csv(path + file)
    dfs[key] = df

## Gene filtering

In [8]:
# Generate gene lists
typelist = 'cell_type_subset'
celltype = 'Tip_Cells'
tumor = 'TumorCell'

adata_pre_celltype = adata_pre[adata_pre.obs[typelist] == celltype].copy()
genelist1 = set(dfs[celltype][celltype].tolist())
genelist2 = set(dfs[celltype][tumor].tolist())

avg_genelist1 = set(adata_pre_celltype.to_df().mean()[adata_pre_celltype.to_df().mean() > 0].index.tolist())
avg_genelist2 = set(adata_pre_tumor.to_df().mean()[adata_pre_tumor.to_df().mean() > 0].index.tolist())

fil_genelist1 = list(genelist1 & avg_genelist1)
fil_genelist2 = list(genelist2 & avg_genelist2)

fil = dfs[celltype][dfs[celltype][celltype].isin(fil_genelist1)]
fil = fil[fil[tumor].isin(fil_genelist2)]
Tip = fil.sort_values(by='Mean Normalized_Weight', ascending = False).reset_index(drop = True)

## calculate_cell_type_percentage: 유전자를 발현하는 세포의수 / 해당 세포 유형의 수

In [12]:
# Calculate ECF
tumor_genes = list(Tip['TumorCell'].unique())
tip_genes = list(Tip['Tip_Cells'].unique())
cell_types_gs = adata_gs.obs['cell_type_subset'].unique()

df_tip_ecf_all_unique = df_tip_ecf_all.drop_duplicates(subset=['Gene'], keep='first').reset_index(drop=True)
df_tumor_ecf_all_unique = df_tumor_ecf_all.drop_duplicates(subset=['Gene'], keep='first').reset_index(drop=True)

# Calculate cell type percentage for TumorCell gene list
tumor_cell_type_percentage_df = calculate_cell_type_percentage(tumor_genes, adata_gs, cell_types_gs, 'cell_type_subset')
tumor_cell_type_percentage_df['Tumor_Total_ECF'] = df_tumor_ecf_all['Total_ECF']

# Calculate cell type percentage for Tip_Cells gene list
tip_cell_type_percentage_df = calculate_cell_type_percentage(tip_genes, adata_gs, cell_types_gs, 'cell_type_subset')
tip_cell_type_percentage_df['Tip_Total_ECF'] = df_tip_ecf_all_unique['Total_ECF']

# Set index and filter results
tumor_cell_type_percentage_df.set_index(tumor_cell_type_percentage_df.columns[0], inplace=True)
filtered_df_tumor = tumor_cell_type_percentage_df[tumor_cell_type_percentage_df['tumor'] == tumor_cell_type_percentage_df.drop('Tumor_Total_ECF', axis=1).max(axis=1)]

tip_cell_type_percentage_df.set_index(tip_cell_type_percentage_df.columns[0], inplace=True)
filtered_df_tip = tip_cell_type_percentage_df[tip_cell_type_percentage_df['Tip_Cells'] == tip_cell_type_percentage_df.drop('Tip_Total_ECF', axis=1).max(axis=1)]

In [72]:
# Extract gene lists
gene_list_tumor = filtered_df_tumor.index.tolist()
gene_list_tip = filtered_df_tip.index.tolist()

tmp = Tip[Tip['TumorCell'].isin(gene_list_tumor)]
tmp = tmp[tmp['Tip_Cells'].isin(gene_list_tip)]
common_pair = tmp.reset_index(drop=True)
common_pair = common_pair[common_pair['Count'] >= 4]
common_pair.reset_index(drop=True).round(4)

,TumorCell,Tip_Cells,Mean Normalized_Weight,Variance Normalized_Weight,Std Dev Normalized_Weight,Median Normalized_Weight,CV Normalized_Weight,Count
0,ARF1,INSR,0.6384,0.0158,0.1258,0.6628,19.7026,10
1,TNFRSF21,APP,0.6259,0.0086,0.0925,0.6220,14.7785,10
2,NCSTN,APP,0.5754,0.0069,0.0831,0.5753,14.4443,10
3,HRAS,INSR,0.5042,0.0063,0.0792,0.5310,15.7046,9
4,GRM7,APP,0.4787,0.0053,0.0727,0.5055,15.1862,9
5,SDC1,COL4A2,0.4670,0.0109,0.1043,0.4509,22.3213,8
6,ITGB8,COL4A1,0.4459,0.0064,0.0801,0.4394,17.9599,8
7,SDC1,COL4A1,0.4392,0.0038,0.0616,0.4411,14.0192,5
8,ITGB8,COL4A2,0.4124,0.0051,0.0712,0.4085,17.2713,6
9,F2,GP9,0.3845,0.0009,0.0303,0.3939,7.8734,6


In [168]:
# 발현값이 0이 아닌 유전자 목록을 추출합니다
non_zero_genes = adata_gs_Tip_Cells.to_df().sum()[adata_gs_Tip_Cells.to_df().sum() != 0].index.tolist()

# TEC 데이터프레임에서 TumorCell 열에 있는 유전자가 non_zero_genes에 포함된 행만 필터링합니다
tmp = Tip_Cells[Tip_Cells['Tip_Cells'].isin(non_zero_genes)]
Tip_Cells = tmp[tmp['TumorCell'].isin(non_zero_genes)]

# 필터링된 데이터프레임 출력
Tip_Cells = Tip_Cells.reset_index(drop=True)
Tip_Cells.head(10)

,TumorCell,Tip_Cells,Mean Normalized_Weight,Variance Normalized_Weight,Std Dev Normalized_Weight,Median Normalized_Weight,CV Normalized_Weight,Count
0,SORBS1,INSR,0.946254,0.011415,0.106842,1.000000,11.291005,10
1,RELN,LRP8,0.913808,0.010509,0.102514,0.963123,11.218329,10
2,LRPAP1,LRP8,0.913108,0.004542,0.067396,0.926928,7.380925,9
3,TAC4,TACR1,0.891795,0.003130,0.055943,0.890395,6.273035,9
4,ARF1,INSR,0.873440,0.007330,0.085617,0.890554,9.802325,10
5,GP6,COL4A3,0.831817,0.005909,0.076869,0.824020,9.241096,8
6,GP6,COL4A2,0.825877,0.006356,0.079724,0.816665,9.653309,8
7,GP6,COL4A1,0.788179,0.004852,0.069658,0.790360,8.837876,8
8,LAT,SYK,0.784056,0.005496,0.074135,0.781243,9.455338,10
9,ITGA2,COL6A5,0.776248,0.002716,0.052120,0.791528,6.714345,9


In [152]:
# 특정 값이 리스트에 있는지 확인
value_to_check = "STAB2"
if value_to_check in non_zero_genes:
    print(f"{value_to_check} is in the list")
else:
    print(f"{value_to_check} is not in the list")

STAB2 is in the list


In [147]:
Tip_Cells_tumor_geneset

['TAC4',
 'RELN',
 'ADCY7',
 'ITGA1',
 'TIE1',
 'DDR1',
 'COL13A1',
 'COL4A3',
 'GP1BA',
 'CD86',
 'ITGB2',
 'FGFR1',
 'LILRB2',
 'CNR1',
 'SELP',
 'LRPAP1',
 'ADM',
 'HRAS',
 'ITGB3',
 'TNFRSF11B',
 'TEK',
 'STAB2',
 'LAT',
 'RPSA',
 'IGF2',
 'GP6',
 'GP9',
 'SIRPA',
 'EGFR',
 'TSHR',
 'CD44',
 'ITGA2',
 'CALM1',
 'SORBS1',
 'ITGA11',
 'SDC1',
 'VIP',
 'CD3G',
 'ITGB1',
 'RACK1',
 'APP',
 'ARF1',
 'ITGAM',
 'ITGA10',
 'F12']

In [131]:
Tip_Cells_tumor_geneset = Tip_Cells['TumorCell'].tolist()
Tip_Cells_tumor_geneset = list(set(Tip_Cells_tumor_geneset))
Tip_Cells_geneset = Tip_Cells['Tip_Cells'].tolist()
Tip_Cells_geneset = list(set(Tip_Cells_geneset))

In [145]:
for gene in Tip_Cells_tumor_geneset:
    print(f"{gene}: {adata_gs_tumor.to_df()[gene].sum()}")

print(len(Tip_Cells_tumor_geneset))

TAC4: 31.763080596923828
RELN: 2.4940667152404785
ADCY7: 56.24870300292969
ITGA1: 98.06460571289062
TIE1: 2.543095827102661
DDR1: 483.135498046875
COL13A1: 8.545530319213867
COL4A3: 133.42506408691406
GP1BA: 3.4376258850097656
CD86: 22.218379974365234
ITGB2: 173.89451599121094
FGFR1: 66.16529846191406
LILRB2: 2.799717426300049
CNR1: 5.792452812194824
SELP: 0.5870054960250854
LRPAP1: 573.1060791015625
ADM: 223.76589965820312
HRAS: 202.82666015625
ITGB3: 12.1261568069458
TNFRSF11B: 18.039751052856445
TEK: 1.2484780550003052
STAB2: 0.0
LAT: 31.83863067626953
RPSA: 2107.72802734375
IGF2: 66.00102996826172
GP6: 3.0656418800354004
GP9: 3.0751864910125732
SIRPA: 34.25025177001953
EGFR: 355.7984619140625
TSHR: 0.0
CD44: 559.008544921875
ITGA2: 278.8961486816406
CALM1: 1644.1676025390625
SORBS1: 42.05267333984375
ITGA11: 0.9280428290367126
SDC1: 659.5999755859375
VIP: 0.5924465656280518
CD3G: 44.56570816040039
ITGB1: 746.1672973632812
RACK1: 2300.13623046875
APP: 844.8216552734375
ARF1: 1185.87

In [146]:
for gene in Tip_Cells_geneset:
    print(f"{gene}: {adata_gs_Tip_Cells.to_df()[gene].sum()}")
    
print(len(Tip_Cells_geneset))

LAMA3: 11.057124137878418
COL4A1: 1436.73779296875
COL4A2: 1215.205810546875
VWF: 1551.2581787109375
PKM: 567.893310546875
LAMC3: 7.151813507080078
COL6A1: 104.76765441894531
LAMA4: 558.978759765625
COL4A3: 2.7139203548431396
COL2A1: 2.031682014465332
ANGPT2: 763.6697998046875
COL6A3: 19.798368453979492
CTLA4: 8.713127136230469
LRP8: 15.289715766906738
HSPG2: 1232.275634765625
TLN1: 309.92742919921875
COL4A5: 8.189508438110352
COL6A5: 2.218614101409912
FGF14: 2.2732386589050293
LAMB1: 312.9258117675781
COL5A2: 53.02064514160156
INSR: 982.8104248046875
GNAI2: 922.1942138671875
COL6A6: 0.2854004204273224
COL9A2: 1.62301504611969
HLA-C: 2102.5927734375
COL1A2: 54.492679595947266
CD93: 1090.904296875
RAMP2: 1375.9051513671875
PTGER2: 0.23933900892734528
F10: 5.346032619476318
TACR1: 17.355602264404297
COL9A3: 7.501163482666016
COL1A1: 37.831111907958984
COL9A1: 0.2854004204273224
CALR: 1044.873291015625
SYK: 7.038078784942627
CDH1: 8.705918312072754
COL6A2: 251.9918670654297
AGTRAP: 146.32

In [170]:
# %matplotlib inline
# %config InlineBackend.figure_format='retina' # mac
#%load_ext autoreload
#%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import copy, os, time
import numpy as np
import scanpy as sc
import anndata

In [171]:
from gseapy import Biomart
bm = Biomart()


In [172]:
# default: Human
names = gp.get_library_name()
names[:10]

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021']